Данная версия является устаревшей, так как новая версия у меня находится на ПК, к которому я в данный момент доступа не имею. Обновленная версия будет загружена на гитхаб в течение выходных.

## Подключение Бибилотек

In [568]:
import numpy as np
import math
import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Разработка среды

In [ ]:
# Гиперпараметры модели
HEIGHT = 9
WIDTH = 9
MINES = 9

In [533]:
class Minesweeper:
    def __init__(self, height = 9, width = 9, mines = 9):
        self.height = height
        self.width = width
        self.nummines = mines
        self.reset()

    def reset(self):
        self.board = np.array([[' ' for i in range(self.width + 2)] for j in range(self.height + 2)])
        self.done = False
        self.reward = 0
        self.moves = [i for i in range(self.height * self.width)] # все доступные ходы
        self.mines = []

    def findmines(self, row, col):
        ''' Возвращает количество мин вокруг текущей клетки '''

        # если в клетке мина, то возвращаем -1
        if row * self.width + col in self.mines:
            return -1
        
        nummines = 0
        for row_offset in  [-1, 0, 1]:
            for col_offset in [-1, 0, 1]:
                if (row + row_offset) * self.width + (col + col_offset) in self.mines:
                    nummines += 1
        return nummines
        
    def step(self, move):
        ''' Получение хода в качестве индекса и открытие клетки'''
        
        if move not in self.moves:
            print('Такой ход сделать невозможно, делается случайный ход')
            move = self.sample()

        self.moves.remove(move)
        # Рандомные позиции для мин на первом ходу
        if len(self.moves) == self.height * self.width - 1:
            self.mines = np.random.choice(self.moves, self.nummines, replace = False)

        row, col = move // self.width, move % self.width
        nummines = self.findmines(row, col)

        if nummines == -1:
            self.board[row + 1, col + 1] = '*'
            self.reward = 0 # -1 вообще
            self.done = True
        else:
            self.board[row + 1, col + 1] = str(nummines)
        # рекурсивный вызов для открытия соседних клеток, если они еще не открыты, в случае если в клетке 0
            if nummines == 0:
                for row_offset in  [-1, 0, 1]:
                    for col_offset in [-1, 0, 1]:
                        if (row_offset, col_offset) == (0, 0):
                            continue
                        if row + row_offset + 1 in [0, self.height + 1] or col + col_offset + 1 in [0, self.width + 1]:
                            continue    # чтобы не выйти за границы поля
                        if (row + row_offset) * self.width + (col + col_offset) in self.moves:
                            self.step((row + row_offset) * self.width + (col + col_offset))

            # проверка на победу (количество оставшихся ходов равно количеству мин)
            if  len(self.moves) == self.nummines:
                self.reward = 1
                self.done = True

    def sample(self):
        return np.random.choice(self.moves)
    
    def get_tensor_state(self):
        ''' one hot encoding для состояния поля '''
        miniboard = self.board[1:-1, 1:-1]
        myboard = [np.array(miniboard == ' ', dtype = int).reshape(self.height, self.width)]

        for i in range(9):
            myboard.append(np.array(miniboard == 'i', dtype = int))
        return tf.convert_to_tensor(myboard)

    def render(self):
        for rows in self.board[1:-1]:
            print('|'.join(rows))
    
    def render_with_mines(self):
        self.mineboard = copy.deepcopy(self.board)
        for mine in self.mines:
            self.board[mine // self.width + 1, mine % self.width + 1] = '* '
        for rows in self.board[1:-1]:
            print('|'.join(rows))

## Случайный агент

In [534]:
def RandomAgent(state, moves):
    '''Возвращает случайное действие из листа ходов'''
    return np.random.choice(moves)

In [535]:
def playgame(env, agent = RandomAgent, numgames = 100, verbose = False):
    total_reward = 0
    for game in range(numgames):
        env.reset()
        while not env.done:
            move = agent(env.get_tensor_state(), env.moves)
            env.step(move)
            if verbose:
                print('Move: ', move // 9, move % 9)
                env.render()
                print()

        total_reward += env.reward
    print(f'количество побед в {numgames} играх: {total_reward}')

In [536]:
playgame(env = Minesweeper(9, 9, 1), agent = RandomAgent, numgames = 100)

количество побед в 100 играх: 79


In [537]:
playgame(env = Minesweeper(9, 9, 2), agent = RandomAgent, numgames = 100)

количество побед в 100 играх: 53


In [538]:
playgame(env = Minesweeper(9, 9, 9), agent = RandomAgent, numgames = 100)

количество побед в 100 играх: 0


## Reinforcment learning

Q(s, a) - ожидаемая награда за ход a в состоянии s.

Необходимо обучить модель, которая будет предсказывать вероятность следующего хода, когда ход не приводит к поражению.

Вход: тензор состояния поля Сапера (где первый слой для доступных ходоа, 2-10 слои для мин, вокруг открытой клетки).

Выход: значения Q(s, a) для всех возможных ходов.

Цель: one-hot encoding для выбранной клетки.

In [541]:
env = Minesweeper(HEIGHT, WIDTH, MINES)

model = keras.Sequential(
    [
        keras.Input(shape = (HEIGHT, WIDTH, 10)),
        layers.Conv2D(32, kernel_size = (3, 3), activation="relu"),
        layers.Conv2D(64, kernel_size = (3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(HEIGHT * WIDTH, activation="softmax"),
    ]
)

In [542]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 7, 7, 32)          2912      
                                                                 
 conv2d_13 (Conv2D)          (None, 5, 5, 64)          18496     
                                                                 
 flatten_6 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 81)                129681    
                                                                 
Total params: 151089 (590.19 KB)
Trainable params: 151089 (590.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [543]:
def get_samples(model, numgames = 100):
    ''' Генерация обучающих данных '''
    x, y = [], []
    env = Minesweeper(HEIGHT, WIDTH, MINES)
    for game in range(numgames):
        env.reset()
        while not env.done:
            cur_state = tf.transpose(env.get_tensor_state(), perm = [1, 2, 0])
            move = RandomAgent(cur_state, env.moves)
            env.step(move)

            # Если накнулись на мину, то не добавляем в обучающие данные
            if env.done and env.reward == 0:
                continue
            else:
                x.append(cur_state)
                y.append([1 if i == move else 0 for i in range(HEIGHT * WIDTH)])
    return tf.convert_to_tensor(x), tf.convert_to_tensor(y)

In [544]:
x_train, y_train = get_samples(model, numgames = 500)

In [545]:
x_train.shape, y_train.shape

(TensorShape([2091, 9, 9, 10]), TensorShape([2091, 81]))

In [546]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

model.fit(x_train, y_train, batch_size = 16, epochs = 100, validation_split = 0.1)

Epoch 1/100
118/118 [==============================] - 0s 2ms/step - loss: 4.3769 - accuracy: 0.0138 - val_loss: 4.3461 - val_accuracy: 0.0333
Epoch 2/100
118/118 [==============================] - 0s 2ms/step - loss: 4.1784 - accuracy: 0.0303 - val_loss: 4.3148 - val_accuracy: 0.0286
Epoch 3/100
118/118 [==============================] - 0s 2ms/step - loss: 3.9786 - accuracy: 0.0585 - val_loss: 4.3880 - val_accuracy: 0.0190
Epoch 4/100
118/118 [==============================] - 0s 2ms/step - loss: 3.7931 - accuracy: 0.0707 - val_loss: 4.5097 - val_accuracy: 0.0381
Epoch 5/100
118/118 [==============================] - 0s 2ms/step - loss: 3.6055 - accuracy: 0.1042 - val_loss: 4.7061 - val_accuracy: 0.0190
Epoch 6/100
118/118 [==============================] - 0s 2ms/step - loss: 3.4310 - accuracy: 0.1334 - val_loss: 4.8789 - val_accuracy: 0.0190
Epoch 7/100
118/118 [==============================] - 0s 2ms/step - loss: 3.2668 - accuracy: 0.1579 - val_loss: 4.9994 - val_accuracy: 0.0190

In [551]:
def NNAgent(state, moves):
    '''возвращает ход с наибольшей вероятностью из доступных ходов''' 
    probs = model(tf.reshape(env.get_tensor_state(), (1, HEIGHT, WIDTH, 10))).numpy().flatten()
    probs[moves] += 1
    return np.argmax(probs)

In [566]:
playgame(env = Minesweeper(HEIGHT, WIDTH, 5), agent = NNAgent, numgames = 100)

количество побед в 100 играх: 5


In [567]:
playgame(env = Minesweeper(HEIGHT, WIDTH, 5), agent = RandomAgent, numgames = 100)

количество побед в 100 играх: 1
